In [58]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from dataset import extract_users_movies_ratings_lists, TripletDataset, extract_matrix_users_movies_ratings
import torch 

#Useful constants
number_of_users, number_of_movies = (10000, 1000)
RANDOM_STATE = 58
DATA_DIR = '../data'


data_pd = pd.read_csv(DATA_DIR+'/data_train.csv')
train_pd, val_pd = train_test_split(data_pd, train_size=0.9, random_state=RANDOM_STATE)


#users_train, movies_train, ratings_train = extract_users_movies_ratings_lists(train_pd)
#users_val, movies_val, ratings_val = extract_users_movies_ratings_lists(val_pd)
matrix_train, _ = extract_matrix_users_movies_ratings(train_pd)
matrix_val, _ = extract_matrix_users_movies_ratings(val_pd)

In [59]:
K=6
u = np.random.rand(number_of_users, K)
v = np.random.rand(number_of_movies, K)



In [73]:
y = matrix_train

In [65]:
def step(u, v, y):
    u_new = np.empty_like(u)
    v_new = np.empty_like(v)

    def f(i, l):
        uv = np.matmul(u[i],v.transpose())
        a = v[:,l] * y[i, :]
        a = a / uv
        b = v[:, l].sum()
    
        return u[i,l]* (a.sum())/b

    for i in range(0, number_of_users):
        for l in range(0, K):
            uv = np.matmul(u[i],v.transpose())
            a = v[:,l] * y[i, :]
            a = a / uv
            b = v[:, l].sum()
        
            u_new[i,l] = f(i,l )
            
        
    u = u_new
    for j in range(0, number_of_movies):
        for l in range(0, K):
            uv = np.matmul(v[j],u.transpose())
            a = u[:,l] * y[:, j]
            a = a / uv
            b = u[:, l].sum()
        
            v_new[j,l] = v[j,l]* (a.sum())/b
            
    return u_new, v_new


In [61]:
def rmse(y, yhat):
    mask = y.astype(np.bool)
    return np.sqrt(np.mean((y-yhat)[mask]**2))

In [67]:
old_score = np.inf
n = 0
while True:
    u, v = step(u, v, matrix_train)
    pred = np.matmul(u,v.transpose())
    val_score = rmse(matrix_val, pred)
    train_score = rmse(matrix_train, pred)

    print(f'Train rmse: {train_score}, val rmse: {val_score}')
    
    if n % 1000 == 0:
        if val_score < old_score:
            np.save('./best_u', u)
            np.save('./best_v', v)
            old_score = val_score
        np.save('./u', u)
        np.save('./v', v)
    n+=1


/tmp/ipykernel_35120/4172897665.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = y.astype(np.bool)


Train rmse: 3.0859329806571574, val rmse: 3.1213895443353676
Train rmse: 3.0857553636200588, val rmse: 3.121201319606884
Train rmse: 3.085586929085274, val rmse: 3.121022923043839
Train rmse: 3.0854270632264806, val rmse: 3.1208537105180474
Train rmse: 3.0852752143734423, val rmse: 3.120693103149719
Train rmse: 3.0851308770399912, val rmse: 3.1205405554875063
Train rmse: 3.084993580377708, val rmse: 3.120395541980746
Train rmse: 3.0848628863549066, val rmse: 3.1202575716013317
Train rmse: 3.0847383857182615, val rmse: 3.1201262117901094
Train rmse: 3.084619690406305, val rmse: 3.120001099542738
Train rmse: 3.0845064328745906, val rmse: 3.1198819315772743
Train rmse: 3.0843982623581714, val rmse: 3.119768433951315


KeyboardInterrupt: 